In [1]:
!pip install --upgrade pythainlp
!pip install --upgrade pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
numba 0.56.4 requires numpy

In [1]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# df=pd.read_csv('dataset_terminal21_pattaya.csv')
url = 'https://raw.githubusercontent.com/ChawinTSR/MADT8101_Customer_Analytics/main/05_Voice%20of%20Customer%20Analytics/Dataset/dataset_terminal21_pattaya_rev1.csv'  # Replace with the actual URL of your CSV file
df = pd.read_csv(url)

In [4]:
df.tail()

,Review_ID,Review
17,18,มันเป็นห้างสรรพสินค้าขนาดใหญ่ที่ทันสมัยแห่งใหม...
18,19,ท่องเที่ยวห้างสรรพสินค้าที่มีธีมพร้อมร้านอาหาร...
19,20,เราพบแบรนด์ที่แตกต่างกันมากทำเลดีมากแต่ละชั้นใ...
20,21,รูปแบบห้างสรรพสินค้าใหม่ที่ดีในเมืองร้านอาหารแ...
21,22,ห้างสรรพสินค้าขนาดใหญ่ที่มีร้านค้ามากมายหลังจา...


In [5]:
stopwords=list(pythainlp.corpus.thai_stopwords())
removed_words=['', ' ','  ','   ', '    ', '     ','\n','/','ว่า','ๆ', '(', ')','พัทยา','การ','มี','ร้าน','ที่','...', '2','-']
screening_words=stopwords + removed_words
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [6]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [7]:
df.tail()

,Review_ID,Review,Review_tokenized
17,18,มันเป็นห้างสรรพสินค้าขนาดใหญ่ที่ทันสมัยแห่งใหม...,"มัน,เป็น,ห้างสรรพสินค้า,ขนาดใหญ่,ที่,ทันสมัย,แ..."
18,19,ท่องเที่ยวห้างสรรพสินค้าที่มีธีมพร้อมร้านอาหาร...,"ท่องเที่ยว,ห้างสรรพสินค้า,ที่,มี,ธีม,พร้อม,ร้า..."
19,20,เราพบแบรนด์ที่แตกต่างกันมากทำเลดีมากแต่ละชั้นใ...,"เรา,พบ,แบรนด์,ที่,แตก,ต่างกัน,มาก,ทำเล,ดีมาก,แ..."
20,21,รูปแบบห้างสรรพสินค้าใหม่ที่ดีในเมืองร้านอาหารแ...,"รูปแบบ,ห้างสรรพสินค้า,ใหม่,ที่,ดี,ใน,เมือง,ร้า..."
21,22,ห้างสรรพสินค้าขนาดใหญ่ที่มีร้านค้ามากมายหลังจา...,"ห้างสรรพสินค้า,ขนาดใหญ่,ที่,มี,ร้านค้า,มากมาย,..."


In [8]:
documents=df['Review_tokenized'].to_list()
texts=[[text for text in doc.split(',')] for doc in documents]
dictionary=gensim.corpora.Dictionary(texts)

In [9]:
print(dictionary.token2id.keys())

dict_keys(['ก\u200b', 'ของ', 'ความสุข', 'คาเฟ่', 'คุณ', 'ช่วย', 'ซุปเปอร์มาร์เก็ต', 'ทรอ', 'ทริป', 'ที่จะ', 'ท่องเที่ยว', 'นิ', 'พัทยา', 'มี', 'ร้านอาหาร', 'สนุกเกอร์', 'สินค้า', 'หลากหลาย', 'ห้างสรรพสินค้า', 'อาทิ', 'อิ', 'อุปกรณ์', 'เกมส์', 'เครื่องประดับ', 'เติมเต็ม', 'เล็ก', 'เสื้อผ้า', 'และ', 'โซน', 'ใน', 'ให้บริการ', '\u200b', ' ', 'กิน', 'ของกิน', 'คือ', 'ตกแต่ง', 'ตาม', 'ถูก', 'ถ่ายรูป', 'ที่', 'ฟู๊ด', 'มา', 'มาก', 'ราคา', 'สวยงาม', 'สำคัญ', 'ห้าง', 'อร่อย', 'อาหาร', 'เทศกาล', 'เยอะ', 'เหมาะสำหรับ', '21', 'centre', 'court', 'food', 'grand', 'pier', 'point', 'กว้างขวาง', 'ครบ', 'คุณภาพ', 'จบ', 'จอด', 'ถึง', 'ทุ่ม', 'พัก', 'ยาก', 'รถ', 'สมัยนี้', 'สะดวก', 'หา', 'ห้า', 'ออก', 'เดียว', 'เปิด', 'เลย', 'โรงหนัง', 'โรงแรม', 'ใหม่', 'ได้', 'ไป', 'ไม่ต้อง', 'ไหน', 'การ', 'จัด', 'ชั้น', 'ดีมาก', 'ต่างประเทศ', 'ถือได้ว่า', 'บรรยากาศ', 'ประเทศ', 'ภายใน', 'ร้านค้า', 'ลงตัว', 'สถานที่', 'สะดวกสบาย', 'ห้องน้ำ', 'เป็น', 'เมือง', 'เหมาะสม', 'แต่ละ', 'แบ่ง', 'แยก', 'โดยเฉพาะ', 'Court', 'Food', '

In [10]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies=[[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [11]:
num_topics = 4
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number fo passes through documents
iterations = 50
eval_every = 1 #Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary
temp = dictionary[0] # This is only to "load" the dicjjtionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize = chunksize, \
                                     alpha='auto', eta='auto', \
                                     iterations=iterations, num_topics=num_topics, \
                                     passes=passes, eval_every=eval_every)

CPU times: user 363 ms, sys: 0 ns, total: 363 ms
Wall time: 386 ms


In [12]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.059121 -0.089870       1        1  42.953538
0      0.071888  0.076412       2        1  25.562810
1     -0.095479 -0.004976       3        1  25.395543
2     -0.035530  0.018434       4        1   6.088108, topic_info=          Term       Freq      Total Category  logprob  loglift
31           ​  12.000000  12.000000  Default  30.0000  30.0000
49       อาหาร  11.000000  11.000000  Default  29.0000  29.0000
40         ที่  28.000000  28.000000  Default  28.0000  28.0000
251  ยอดเยี่ยม   3.000000   3.000000  Default  27.0000  27.0000
32              29.000000  29.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
53          21   0.484950   3.471854   Topic4  -4.6268   0.8304
49       อาหาร   1.198927  11.853197   Topic4  -3.7217   0.5077
40         ที่   1.392361  28.774751   Topic4  -3.5721  -0.2297
29          ใน   0.501699   8.504176   Topic4  -4.5929  -0.0315
27         และ   0.493385  15.156986   Topic4  -4.6096  -0.6261

[232 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
32        1  0.756355      
32        2  0.034380      
32        3  0.206279      
53        1  0.576061    21
107       1  0.872249  Food
...     ...       ...   ...
82        1  0.872249    ไป
143       1  0.659972   ไม่
143       2  0.439981   ไม่
159       2  0.725629     ๆ
31        2  0.932493     ​

[222 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3])

In [13]:
model.show_topic(1)

[('ที่', 0.051681016),
 ('และ', 0.035430685),
 (' ', 0.028221445),
 ('มาก', 0.018180283),
 ('ร้านค้า', 0.015098291),
 ('ขนาดใหญ่', 0.014714023),
 ('ทันสมัย', 0.014712427),
 ('ใน', 0.01458388),
 ('ได้', 0.014526727),
 ('มากมาย', 0.014518224)]

In [14]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [15]:
df.tail()

,Review_ID,Review,Review_tokenized,topics,score
17,18,มันเป็นห้างสรรพสินค้าขนาดใหญ่ที่ทันสมัยแห่งใหม...,"มัน,เป็น,ห้างสรรพสินค้า,ขนาดใหญ่,ที่,ทันสมัย,แ...",1,0.998390
18,19,ท่องเที่ยวห้างสรรพสินค้าที่มีธีมพร้อมร้านอาหาร...,"ท่องเที่ยว,ห้างสรรพสินค้า,ที่,มี,ธีม,พร้อม,ร้า...",0,0.993355
19,20,เราพบแบรนด์ที่แตกต่างกันมากทำเลดีมากแต่ละชั้นใ...,"เรา,พบ,แบรนด์,ที่,แตก,ต่างกัน,มาก,ทำเล,ดีมาก,แ...",1,0.993760
20,21,รูปแบบห้างสรรพสินค้าใหม่ที่ดีในเมืองร้านอาหารแ...,"รูปแบบ,ห้างสรรพสินค้า,ใหม่,ที่,ดี,ใน,เมือง,ร้า...",1,0.995191
21,22,ห้างสรรพสินค้าขนาดใหญ่ที่มีร้านค้ามากมายหลังจา...,"ห้างสรรพสินค้า,ขนาดใหญ่,ที่,มี,ร้านค้า,มากมาย,...",1,0.996943
